In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from harvai.data import get_clean_preproc_data

In [7]:
data = get_clean_preproc_data()

In [9]:
data.head()

,article_base,article_number,article_content,article_lowered,article_wo_numbers,article_wo_punctuation,article_wo_stopwords,article_tfidf_format,article_lemmatized
0,Article L110-1 Pour l application du présent...,Article L110-1,"Pour l application du présent code, les termes...","pour l application du présent code, les termes...","pour l application du présent code, les termes...",pour l application du présent code les termes...,"[application, présent, code, termes, ci, après...",application présent code termes ci après sens ...,application présent code terme ci après sens d...
1,Article L110-2 La définition des voiries nat...,Article L110-2,"La définition des voiries nationales, départem...","la définition des voiries nationales, départem...","la définition des voiries nationales, départem...",la définition des voiries nationales départem...,"[définition, voiries, nationales, départementa...",définition voiries nationales départementales ...,définition voirie nationale départementale com...
2,Article L110-3 Les routes à grande circulati...,Article L110-3,"Les routes à grande circulation, quelle que so...","les routes à grande circulation, quelle que so...","les routes à grande circulation, quelle que so...",les routes à grande circulation quelle que so...,"[routes, grande, circulation, quelle, apparten...",routes grande circulation quelle appartenance ...,route grand circulation quel appartenance doma...
3,Article L121-1 Le conducteur d un véhicule e...,Article L121-1,Le conducteur d un véhicule est responsable pé...,le conducteur d un véhicule est responsable pé...,le conducteur d un véhicule est responsable pé...,le conducteur d un véhicule est responsable pé...,"[conducteur, véhicule, responsable, pénalement...",conducteur véhicule responsable pénalement inf...,conducteur véhicule responsable pénalement inf...
4,Article L121-2 Par dérogation aux dispositio...,Article L121-2,Par dérogation aux dispositions de l article L...,par dérogation aux dispositions de l article l...,par dérogation aux dispositions de l article l...,par dérogation aux dispositions de l article l...,"[dérogation, dispositions, article, titulaire,...",dérogation dispositions article titulaire cert...,dérogation disposition article titulaire certi...


In [ ]:
question_df = pd.read_csv("")